# 역전파 과정 코딩하기

신경망 모형에서는 역전파 과정을 통해 파라미터를 업데이트하여 성능을 높입니다.  
파이토치 프레임워크에서는 손실함수를 정의하고 `loss.backward()` 와 같은 메서드로 쉽게 역전파가 가능하지만, 이해가 부족했고 직접 해볼 필요성을 느껴서 파이토치의 텐서를 이용하여 역전파 과정을 짜보고자 합니다.

## 참고

[모두를 위한 딥러닝 시즌2](https://www.youtube.com/watch?v=B3VG-TeO9Lk&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=7)

[cs231n Stanford University](https://www.youtube.com/watch?v=i94OvYb6noo&t=989s)

### 과정
1. input layer에 feature의 수만큼 노드를 생성
2. N 개의 hidden layer에 직접 노드를 생성
3. 각 hidden layer를 통과할 때마다 활성화 함수를 거침
4. output layer에 얻고 싶은 결과에 따라 노드를 생성
5. chain rule에 의해 gradient 값을 구하고 gradient descent 방법으로 파라미터 업데이트
6. 위의 과정 반복

In [1]:
import torch

In [2]:
torch.__version__

'1.9.0'

In [3]:
# input layer 
X = torch.Tensor([[0,0], [0,1], [1,0], [1,1]])
y = torch.Tensor([[0], [1], [1], [0]])

# weight & bias
w1 = torch.Tensor(2,2)
b1 = torch.Tensor(2)
w2 = torch.Tensor(2,1)
b2 = torch.Tensor(1)

# Activation function
def sigmoid(x):
    return 1.0 / (1.0+torch.exp(-x))

# Activation function prime
def sigmoid_p(x):
    return sigmoid(x) * (1-sigmoid(x))

# Loss function (binary cross entropy)
def CE(y, y_pred):
    return -torch.mean(y * torch.log(y_pred) + (1-y) * torch.log(1-y_pred))

**Sigmoid**
$$ sigmoid(x) = \frac{1}{1+e^{-x}}$$  
$$ \frac{d}{dx}sigmoid(x) = \frac{1}{1+e^{-x}} \times \frac{e^{-x}}{1+e^{-x}}$$ 

**Binary Cross Entropy**
$$ Loss = -\frac{1}{N}\Sigma{y_i\cdot log(p(y_i)) + (1-y_i)\cdot log(1-p(y_i))} $$



In [236]:
l1 = torch.add(torch.matmul(X,w1),b1) # w1x + b1
a1 = sigmoid(l1)
l2 = torch.add(torch.matmul(a1,w2),b2)
y_pred = sigmoid(l2)

In [237]:
loss = CE(y,y_pred)
loss

tensor(nan)

In [200]:
loss.item()

1.8333158493041992

In [187]:
(y/y_pred) - (1-y)/(1-y_pred)

tensor([[-29.3014],
        [  1.0244],
        [  1.0244],
        [-49.7724]])

In [188]:
d_loss = (y_pred - y) / (y_pred * (1 - y_pred) + 1e-7)
d_loss * sigmoid_p(l2)

tensor([[ 0.9659],
        [-0.0238],
        [-0.0238],
        [ 0.9799]])

In [189]:
(y_pred - y) / (y_pred * (1 - y_pred) + 1e-7)

tensor([[29.3014],
        [-1.0244],
        [-1.0244],
        [49.7721]])

In [190]:
((y/y_pred) - (1-y)/(1-y_pred)) * sigmoid_p(l2)

tensor([[-0.9659],
        [ 0.0238],
        [ 0.0238],
        [-0.9799]])

In [191]:
    d_loss = (y/y_pred) - (1-y)/(1-y_pred)
    d_l2 = sigmoid_p(l2) * d_loss

In [203]:
EPOCH = 10000
lr = 0.01
for epoch in range(1,EPOCH+1):
    # 순전파 
    l1 = torch.add(torch.matmul(X,w1),b1) # w1x + b1
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1,w2),b2)
    y_pred = sigmoid(l2)
    
    loss = CE(y,y_pred)
    print(loss.item())
    #역전파
    
    d_loss = (y/y_pred + 1e-7) - (1-y)/(1-y_pred+ 1e-7)
    
    # layer 2
    d_l2 = sigmoid_p(l2) * d_loss
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_l2) # 0,1 -> 행과 열 변경
    
    # layer 1
    d_a1 = torch.matmul(d_l2, torch.transpose(w2, 0, 1))
    d_l1 = sigmoid_p(l1) * d_a1
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_l1)
    
    # gradient descent
    w1 -= lr * d_w1
    b1 -= lr * torch.mean(d_b1,0)
    w2 -= lr * d_w2
    b2 -= lr * torch.mean(d_b2,0)
    
    # result print
    if epoch % 500 == 0:
        print(f"EPOCH : {epoch} / 10000 ({epoch*100/10000:.0f}%) \t Loss : {loss.item()}")

1.8333158493041992
1.8499536514282227
1.866657018661499
1.8834228515625
1.900252342224121
1.9171439409255981
1.9340916872024536
1.95110023021698
1.9681645631790161
1.9852828979492188
2.002453327178955
2.019679307937622
2.0369515419006348
2.0542781352996826
2.0716474056243896
2.0890674591064453
2.1065313816070557
2.124039649963379
2.141589641571045
2.159184455871582
2.1768178939819336
2.194495677947998
2.212207794189453
2.2299606800079346
2.247748374938965
2.2655739784240723
2.2834315299987793
2.3013267517089844
2.3192522525787354
2.3372111320495605
2.3552045822143555
2.3732266426086426
2.39127779006958
2.409360885620117
2.427467107772827
2.445611000061035
2.463773250579834
2.4819674491882324
2.5001883506774902
2.518432140350342
2.536698818206787
2.5549941062927246
2.5733115673065186
2.5916531085968018
2.6100172996520996
2.6283998489379883
2.6468091011047363
2.6652398109436035
2.6836893558502197
2.7021565437316895
2.720658779144287
2.73915958404541
2.757694959640503
2.7762460708618164
2

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
EPOCH : 3500 / 10000 (35%) 	 Loss : nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
EPOCH : 5500 / 10000 (55%) 	 Loss : nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
EPOCH : 10000 / 10000 (100%) 	 Loss : nan


# 역전파 과정 스스로 짜보기


역전파 과정을 아무것도 보지 않고 스스로 짜보았습니다. 회귀 문제를 푸는 과정이며, **활성화 함수는 ReLU**를 **손실함수는 MSE**를 사용했습니다.

In [19]:
torch.manual_seed(1234)
import torch.nn.functional as F
import torch.nn as nn

In [15]:
# 초기 데이터 정의
X = torch.randn(30,50,
               requires_grad = False,
               dtype = torch.float) 

B = torch.randn(50,1,
               requires_grad = False,
               dtype = torch.float)

w1 = torch.randn(50, 30,
               requires_grad = False,
               dtype = torch.float)

w2 = torch.randn(30,1,
               requires_grad = False,
               dtype = torch.float)

y =  X.mm(B) + torch.randn(30,1,
               requires_grad = False,
               dtype = torch.float)

# 손실함수
def MSE(y, y_pred):
    return 0.5 * torch.mean((torch.sub(y-y_pred))^2)

def MSE_p(y, y_pred):
    return -torch.mean(torch.sub(y-y_pred))

In [20]:
y

tensor([[ -5.2334],
        [  3.0594],
        [ -4.4701],
        [ 14.8281],
        [ -4.4038],
        [ -7.2491],
        [ -7.7360],
        [ -4.7171],
        [-15.2053],
        [  0.7095],
        [ 12.2014],
        [ -2.1960],
        [ -6.8417],
        [-11.4096],
        [  1.8938],
        [  5.9288],
        [  8.3638],
        [ -5.6630],
        [  4.4595],
        [  2.3244],
        [ -5.9600],
        [ -4.2309],
        [  8.0168],
        [ -3.8345],
        [  9.5452],
        [ -7.8699],
        [  3.3523],
        [  2.0816],
        [-11.7269],
        [ -5.0930]])

In [21]:
EPOCH = 5000
lr = 0.01

l1 = torch.matmul(X, w1) # (30, 30)
a1 = F.relu(l1) # (30, 30)
l2 = torch.matmul(a1, w2) # (30, 1)
y_pred = l2 # (30, 1)

loss = MSE(y,y_pred)

d_loss = torch.mean(torch.sub(y_pred-y))
d_l2 = d_loss

TypeError: sub() received an invalid combination of arguments - got (Tensor), but expected (Tensor input, Tensor other, *, Number alpha, Tensor out)